# Zahra Khatibi - 610398119 - Compressing Images using Cosine Transform

In [9]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from itertools import groupby
import sys

# Quantization Matrix

In [10]:
quantization_matrix = np.array([[16, 11, 10, 16, 24, 40, 51, 61],[12, 12, 14, 19, 26, 58, 60, 55],
                                [14, 13, 16, 24, 40, 57, 69, 56],[14, 17, 22, 29, 51, 87, 80, 62],
                                [18, 22, 37, 56, 68, 109, 103, 77],[24, 35, 55, 64, 81, 104, 113, 92],
                                [49, 64, 78, 87, 103, 121, 120, 101],[72, 92, 95, 98, 112, 100, 103, 99]])

# Discrete cosine transform

In [11]:
CN = np.zeros((8, 8))        
for i in range(8):
    for j in range(8):
        if i==0:
            CN[i, j] = 1/np.sqrt(8)
        else:
            CN[i, j] = np.sqrt(2/8)*np.cos((np.pi*i*(2*j+1))/(2*8))
            
def cos_trans(matrix): 
    m1 = CN.dot(matrix)
    m2 = m1.dot(CN.T)
    return m2

def quanting(matrix):                   ######### this part is lossy #########
    m1 = matrix/quantization_matrix
    return np.round(m1)

def dequant(matrix):        
    return matrix*quantization_matrix

def cos_trans_inv(matrix):
    m1 = CN.T.dot(matrix)
    m2 = m1.dot(CN)
    return m2

def divide_and_set(matrix,func):      # divide to 8*8 group
    N, M = matrix.shape
    res_matrix = np.zeros((N, M))
    for i in range(0, N, 8):
        for j in range(0, M, 8):
            chunk = matrix[i:i+8, j:j+8]
            res_chunk = func(chunk)
            res_matrix[i:i+8, j:j+8] = res_chunk
    return res_matrix
 
######################### compression and decompression ########################### 

def compression(matrix):
    flat_matrix = matrix.flatten()
    encoded = [(len(list(group)), val) for val, group in groupby(flat_matrix)]
    return encoded

def decompression(matrix):
    res=[]
    for count,value in matrix:
        res += [value]*count
    return np.array(res).reshape((N,M))

# Open Image File

In [12]:
raw_image = Image.open("D:\IMG_7475.JPG")
image = np.array(raw_image)

# mazrab 8 konim baraye sadegi
N,M,p = image.shape
N -= N%8
M -= M%8
image = image[:N,:M]

# 1: Separate Color

In [13]:
red_matrix = image[:,:,0].astype(int)
green_matrix = image[:,:,1].astype(int)
blue_matrix = image[:,:,2].astype(int)
# scaling
red_matrix -= 127
green_matrix -= 127
blue_matrix -= 127

# 2: Transform

In [14]:
red_dct = divide_and_set(red_matrix,cos_trans)
green_dct = divide_and_set(green_matrix,cos_trans)
blue_dct = divide_and_set(blue_matrix,cos_trans)

# 3: Quantize

In [15]:
red_q = divide_and_set(red_dct,quanting)
green_q = divide_and_set(green_dct,quanting)
blue_q = divide_and_set(blue_dct,quanting)

# 4: compress

In [16]:
red_comp = compression(red_q)
green_comp = compression(green_q)
blue_comp = compression(blue_q)

# Print Result

In [17]:
print(f'Zeros rate is: {int((np.size(red_q)-np.count_nonzero(red_q))*100/np.size(red_q))}')
print(f'Compression rate is: {100-int(sys.getsizeof(red_comp)*100/sys.getsizeof(red_matrix))}')

Zeros rate is: 93
Compression rate is: 79


# Decoding Reverse

# 4': Decompress

In [18]:
red_decomp = decompression(red_comp)
green_decomp = decompression(green_comp) 
blue_decomp = decompression(blue_comp) 

# 3': Dequantize

In [19]:
red_dq = divide_and_set(red_decomp,dequant)
green_dq = divide_and_set(green_decomp,dequant)
blue_dq = divide_and_set(blue_decomp,dequant)

# 2': Inverse Transform


In [20]:
red_matrix_ext = divide_and_set(red_dq,cos_trans_inv)
green_matrix_ext = divide_and_set(green_dq,cos_trans_inv)
blue_matrix_ext = divide_and_set(blue_dq,cos_trans_inv)

# 1' : Mix Color

In [21]:
red_matrix_ext2 = (red_matrix_ext+127).astype(np.uint8)
green_matrix_ext2 = (green_matrix_ext+127).astype(np.uint8)
blue_matrix_ext2 = (blue_matrix_ext+127).astype(np.uint8)
rgb = np.dstack((red_matrix_ext2,green_matrix_ext2,blue_matrix_ext2))
revived_image = Image.fromarray(rgb, "RGB")

# show image

## raw image

In [29]:
raw_image

<img src="qq.png" alt="Example Image" width="70%">

## revived image

In [30]:
revived_image

<img src="pp.png" alt="Example Image" width="70%">

## Distance of images

In [24]:
print(f'Distance of images: {np.sum((red_matrix_ext-red_matrix)**2)}')

Distance of images: 68547109.58408792


# All result

In [25]:
print(f'Zeros rate is: {int((np.size(red_q)-np.count_nonzero(red_q))*100/np.size(red_q))}') 
print(f'Compression rate is: {100-int(sys.getsizeof(red_comp)*100/sys.getsizeof(red_matrix))}')
print(f'Distance of images: {np.sum((red_matrix_ext-red_matrix)**2)}') # use sqrt(sigma |ui|^2) ###### norm 2 #####

Zeros rate is: 93
Compression rate is: 79
Distance of images: 68547109.58408792
